In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import time
import random
import calendar

In [ ]:
def DD(array):
    n = len(array)
    DD = np.zeros(n)
    prevmax = 0
    for i in range(n):
        if array[i] > prevmax:
            prevmax = array[i]
        DD[i] = max(0,prevmax - array[i])
    return DD

In [ ]:
HO = pd.read_csv('HO-5min.asc',index_col = 0).iloc[:,:5]
HO.index = pd.to_datetime(HO.index)
HO.iloc[:,1:] = HO.iloc[:,1:].multiply(100)

XB = pd.read_csv('XB-5min.asc',index_col = 0).iloc[:,:5]
XB.index = pd.to_datetime(XB.index)
XB.iloc[:,1:] = XB.iloc[:,1:].multiply(100)

In [ ]:
data = XB[(XB.index > datetime(2012,1,1)) & (XB.index < datetime(2020,1,1))]

In [ ]:
df = pd.read_csv('XB_8yrs_TradeTables_4Ytrain_4Qtest_EnhanceRS.csv',index_col = 0)
df_EC = pd.read_csv('XB_8yrs_EC_4Ytrain_4Qtest_EnhanceRS.csv',index_col = 0)

In [ ]:
Drawdown = DD(np.array(df_EC.Equity))

In [ ]:
Net_Equity = df.Profit.sum() + 100000
Net_Profit = df.Profit.sum()
Worst_Drawdown = max(Drawdown)
Net_Profit_To_Worst_Drawdown = Net_Profit/Worst_Drawdown*100
Avg_Drawdown = np.mean(Drawdown)
Gross_Gain = df.Profit[df.Profit >= 0].sum()
Gross_Loss = -df.Profit[df.Profit < 0].sum()
Profit_Factor = Gross_Gain/Gross_Loss
Trade_Count = len(df)
Long_Trade_Count = sum(df.StartPos == 'long')
Short_Trade_Count = sum(df.StartPos == 'short')
Percent_Winners = sum(df.Profit >= 0)/Trade_Count*100
Percent_Losers = sum(df.Profit < 0)/Trade_Count*100
Winner_Losers_Ratio = Percent_Winners/Percent_Losers*100

In [ ]:
Winner = []
Loser = []
Profit = np.array(df.Profit)
EntryPrice = np.array(df.EntryPrice)
ClosePrice = np.array(df.ClosePrice)
StartPos = np.array(df.StartPos)
for i in range(Trade_Count):
    if Profit[i] >= 0:
        if StartPos[i] == 'long':
            Winner.append((ClosePrice[i]-EntryPrice[i])/EntryPrice[i])
        else:
            Winner.append((EntryPrice[i]-ClosePrice[i])/EntryPrice[i])
    else:
        if StartPos[i] == 'long':
            Loser.append((EntryPrice[i]-ClosePrice[i])/EntryPrice[i])
        else:
            Loser.append((ClosePrice[i]-EntryPrice[i])/EntryPrice[i])

In [ ]:
Avg_Winner = np.mean(Winner)*100
Avg_Loser = np.mean(Loser)*100
Avg_Winner_to_Avg_Loser = Avg_Winner/Avg_Loser*100
Best_Winner = max(Winner)*100
Worst_Loser = max(Loser)*100
Best_Winner_To_Worst_Loser = Best_Winner/Worst_Loser*100

In [ ]:
index_i = np.zeros(Trade_Count)
index_j = np.zeros(Trade_Count)
Date = np.array(data.index)
Time = np.array(data.Time)
n = len(data)
i = 0
start = time.time()
for trade in range(Trade_Count):
    for index in range(i,n):
        if str(Date[index])[:10] == df.StartDate[trade] and Time[index] == df.StartTime[trade]:
            index_i[trade] = index
        if str(Date[index])[:10] == df.EndDate[trade] and Time[index] == df.EndTime[trade]:
            index_j[trade] = index
            i = index
            pass
end = time.time()
end - start

In [ ]:
df['Bars'] = index_j-index_i

In [ ]:
Avg_Bars_In_Trade = np.mean(df.Bars)
Avg_Bars_In_Win_Trade = np.mean(df.Bars[df.Profit >= 0])
Avg_Bars_In_Lose_Trade = np.mean(df.Bars[df.Profit < 0])

In [ ]:
rt = np.diff(np.log(df_EC.Equity))
avg_ror = np.mean(rt)*len(rt)/20*100
avg_std = np.std(rt)*np.sqrt(len(rt))/20*100
Sharpe_Ratio = avg_ror/avg_std

In [ ]:
n = 50
file = open("copy.txt", "w") 
file.write("Performance Results\n\n")
file.write('Time Series: XB/XB-5min.asc\n\n')
file.write('Basic Channel System\n')
file.write('-'*100)
file.write('\n'*3)
file.write('Net Equity ' + '-'*(n-len('Net Equity ')) + ' $' + str(round(Net_Equity,2))+'\n')
file.write('Net Profit ' + '-'*(n-len('Net Profit ')) + ' $' + str(round(Net_Profit,2))+'\n')
file.write('Net Profit To Worst Drawdown ' + '-'*(n-len('Net Profit To Worst Drawdown ')) + ' '
           + str(round(Net_Profit_To_Worst_Drawdown,2))+'%\n')
file.write('Average Loser ' + '-'*(n-len('Average Loser ')) + ' -' + str(round(Avg_Loser,2))+'%\n')
file.write('Average Winner ' + '-'*(n-len('Average Winner ')) + ' ' + str(round(Avg_Winner,2))+'%\n')
file.write('Average Winner To Average Loser ' + '-'*(n-len('Average Winner To Average Loser ')) + ' '
           + str(round(Avg_Winner_to_Avg_Loser,2))+'%\n')
file.write('Gross Gain ' + '-'*(n-len('Gross Gain ')) + ' $' + str(round(Gross_Gain,2))+'\n')
file.write('Gross Loss ' + '-'*(n-len('Gross Loss ')) + ' ($' + str(round(Gross_Loss,2))+')\n')
file.write('Profit Factor ' + '-'*(n-len('Profit Factor ')) + ' ' + str(round(Profit_Factor,2))+'\n')
file.write('Trade Count ' + '-'*(n-len('Trade Count ')) + ' ' + str(round(Trade_Count,2))+'\n')
file.write('Long Trade Count ' + '-'*(n-len('Long Trade Count ')) + ' ' + str(round(Long_Trade_Count,2))+'\n')
file.write('Short Trade Count ' + '-'*(n-len('Short Trade Count ')) + ' ' + str(round(Short_Trade_Count,2))+'\n')
file.write('Average Drawdown ' + '-'*(n-len('Average Drawdown ')) + ' ($' + str(round(Avg_Drawdown,2))+')\n')
file.write('Worst Drawdown ' + '-'*(n-len('Worst Drawdown ')) + ' ($' + str(round(Worst_Drawdown,2))+')\n')
file.write('Percent Winners ' + '-'*(n-len('Percent Winners ')) + ' ' + str(round(Percent_Winners,2))+'%\n')
file.write('Percent Losers ' + '-'*(n-len('Percent Losers ')) + ' ' + str(round(Percent_Losers,2))+'%\n')
file.write('Best Winner ' + '-'*(n-len('Best Winner ')) + ' ' + str(round(Best_Winner,2))+'%\n')
file.write('Worst Loser ' + '-'*(n-len('Worst Loser ')) + ' -' + str(round(Worst_Loser,2))+'%\n')
file.write('Best Winner To Worst Loser ' + '-'*(n-len('Best Winner To Worst Loser ')) + ' '
           + str(round(Best_Winner_To_Worst_Loser,2))+'%\n')
file.write('Winners Losers Ratio ' + '-'*(n-len('Winners Losers Ratio ')) + ' ' + str(round(Winner_Losers_Ratio,2))+'%\n')
file.write('Sharpe Ratio ' + '-'*(n-len('Sharpe Ratio ')) + ' ' + str(round(Sharpe_Ratio,2))+'\n')
file.write('Average RoR ' + '-'*(n-len('Average RoR ')) + ' ' + str(round(avg_ror,2))+'%\n')
file.write('Average Std Dev ' + '-'*(n-len('Average Std Dev ')) + ' ' + str(round(avg_std,2))+'%\n')
file.write('Avg Bars in Trade ' + '-'*(n-len('Avg Bars in Trade ')) + ' ' + str(round(Avg_Bars_In_Trade))+'\n')
file.write('Avg Bars in Winning Trade ' + '-'*(n-len('Avg Bars in Winning Trade ')) + ' '
           + str(round(Avg_Bars_In_Win_Trade))+'\n')
file.write('Avg Bars in Losing Trade ' + '-'*(n-len('Avg Bars in Losing Trade ')) + ' '
           + str(round(Avg_Bars_In_Lose_Trade))+'\n\n')
file.write('-'*100 + '\n\n')
n = 25
file.write('Slippage ' + ' '*(n-len('Slippage ')) + str(91) +'\n')
file.write('Series Date Start ' + ' '*(n-len('Series Date Start ')) + str(data.index[0])[:10] +'\n')
file.write('Series Date End ' + ' '*(n-len('Series Date End ')) + str(data.index[-1])[:10] +'\n')
file.close() 